## Finetune Llama-2-7b using QLora on a Google colab

Welcome to this Google Colab notebook that shows how to fine-tune the recent Llama-2-7b model on a single Google colab and turn it into a chatbot

We will leverage PEFT library from Hugging Face ecosystem, as well as QLoRA for more memory efficient finetuning

## Setup

Run the cells below to setup and install the required libraries. For our experiment we will need `accelerate`, `peft`, `transformers`, `datasets` and TRL to leverage the recent [`SFTTrainer`](https://huggingface.co/docs/trl/main/en/sft_trainer). We will use `bitsandbytes` to [quantize the base model into 4bit](https://huggingface.co/blog/4bit-transformers-bitsandbytes). We will also install `einops` as it is a requirement to load Falcon models.

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

Connect with HF account

Follow the Instructions to generate huggingfaces token with write access

Create an account, if you don't have account in Hugging Face. If it is already present then

Go to Profile -> Access Tokens -> Create a token with write access -> Copy it and use it for login

In [ ]:
# login with hugging face token
notebook_login()

In [ ]:
!nvidia-smi

Sun May 26 23:06:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# loading and reading values in from whatever our dataset is called
with open("ecommerce-faq.json") as json_file:
    data = json.load(json_file)

In [ ]:
# Will need to change this based on dataset
pprint(data["questions"][0], sort_dicts=False)

{'question': 'How can I create an account?',
 'answer': "To create an account, click on the 'Sign Up' button on the top "
           'right corner of our website and follow the instructions to '
           'complete the registration process.'}


In [ ]:
pprint(data["questions"][1], sort_dicts=False)

{'question': 'What payment methods do you accept?',
 'answer': 'We accept major credit cards, debit cards, and PayPal as payment '
           'methods for online orders.'}


In [ ]:
pprint(data["questions"][3], sort_dicts=False)

{'question': 'What is your return policy?',
 'answer': 'Our return policy allows you to return products within 30 days of '
           'purchase for a full refund, provided they are in their original '
           'condition and packaging. Please refer to our Returns page for '
           'detailed instructions.'}


In [ ]:
# reading dataset.json for Q&A
with open("dataset.json", "w") as f:
    json.dump(data["questions"], f)

In [ ]:
pd.DataFrame(data["questions"]).head()

,question,answer
0,How can I create an account?,"To create an account, click on the 'Sign Up' b..."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and..."
2,How can I track my order?,You can track your order by logging into your ...
3,What is your return policy?,Our return policy allows you to return product...
4,Can I cancel my order?,You can cancel your order if it has not been s...


we are using a sharded model the advantage of using sharded model is when you combine that with accelerate like for example in this case the model has been sharded into multiple pieces like more than like 14 pieces approximately so that helps accelerate to take a particular piece and then move it to different parts of the memory sometimes GPU memory sometimes CPU memory so that helps you

load a very large model and also fine tune a large model in a smaller amount of memory that you have caught that's why we are using the sharded model

### Changing the quantization type

The 4bit integration comes with 2 different quantization types: FP4 and NF4. The NF4 dtype stands for Normal Float 4 and is introduced in the [QLoRA paper](https://arxiv.org/abs/2305.14314)

YOu can switch between these two dtype using `bnb_4bit_quant_type` from `BitsAndBytesConfig`. By default, the FP4 quantization is used.

In [ ]:
# Load Model & Tokenizer

MODEL_NAME = "realshyfox/sharded-Llama-3-8B" # Change this with specific model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  #
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    quantization_config=bnb_config,
)

# tockenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/317 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# helper function to print number of trainable parameters
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params: {trainable_params} || All params: {all_param} || Trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

**LoraConfig** allows you to control how LoRA is applied to the base model through the following parameters: \

**r**: the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters. \

**target_modules**: The modules (for example, attention blocks) to apply the LoRA update matrices. \

**alpha** : LoRA scaling factor. \

**bias**: Specifies if the bias parameters should be trained. Can be 'none', 'all' or 'lora_only'. \

**modules_to_save**: List of modules apart from LoRA layers to be set as trainable and saved in the final checkpoint. These typically include model’s custom head that is randomly initialized for the fine-tuning task.
**layers_to_transform**: List of layers to be transformed by LoRA. If not specified, all layers in target_modules are transformed. \

**layers_pattern**: Pattern to match layer names in target_modules, if layers_to_transform is specified. By default PeftModel will look at common layer pattern (layers, h, blocks, etc.), use it for exotic and custom models. \

**rank_pattern**: The mapping from layer names or regexp expression to ranks which are different from the default rank specified by r. \

**alpha_pattern**: The mapping from layer names or regexp expression to alphas which are different from the default alpha specified by lora_alpha.

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

Trainable params: 27262976 || All params: 4567863296 || Trainable%: 0.5968430803057028


In [ ]:
# Inference Before Training

prompt = f"""
: How can I create an account?
:
""".strip()
print(prompt)

: How can I create an account?
:


In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 80
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
# generation configurations
generation_config

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128001,
  "max_length": 4096,
  "max_new_tokens": 80,
  "pad_token_id": 128001,
  "temperature": 0.7,
  "top_p": 0.7
}

In [ ]:
%%time
# Specify the target device for model execution, typically a GPU.
device = "cuda:0"

# Tokenize the input prompt and move it to the specified device.
encoding = tokenizer(prompt, return_tensors="pt").to(device)

# Run model inference in evaluation mode (inference_mode) for efficiency.
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )


# Decode the generated output and print it, excluding special tokens.
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

: How can I create an account?
: I have an account, but I forgot my password. How can I reset it?
: How can I change my password?
: How can I change my e-mail address?
: How can I change my first name?
: How can I change my last name?
: How can I change my phone number?
: How can I change my mailing address?
: How can I change my credit card information
CPU times: user 11.1 s, sys: 793 ms, total: 11.9 s
Wall time: 14.8 s


#Build HuggingFace Dataset format

In [ ]:
data = load_dataset("json", data_files="dataset.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 79
    })
})

In [ ]:
data["train"][0]

{'question': 'How can I create an account?',
 'answer': "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."}

In [ ]:
def generate_prompt(data_point):
    return f"""
: {data_point["question"]}
: {data_point["answer"]}
""".strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
data

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 79
})

In [ ]:
OUTPUT_DIR = "experiments"

In [ ]:
# training
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.987200
2,1.750800
3,1.787100
4,1.968100
5,2.056700
6,1.821000
7,1.779400
8,1.656500
9,1.760000
10,1.466700


TrainOutput(global_step=80, training_loss=0.9092550650238991, metrics={'train_runtime': 466.6794, 'train_samples_per_second': 0.686, 'train_steps_per_second': 0.171, 'total_flos': 680337317117952.0, 'train_loss': 0.9092550650238991, 'epoch': 4.050632911392405})

I trained it for 100 epochs, and as you can observe, the loss consistently decreases, indicating room for further improvement. Consider extending the training to a higher number of epochs for potential enhancements

## save the model

In [ ]:
model.save_pretrained("trained-model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Push the model in Hugging face

In [ ]:
model.push_to_hub("jasonweber99/Llama3-8b-qlora-test") # Change with your name

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jasonweber99/Llama3-8b-qlora-test/commit/701fc782c5b9f0e23b444f2f0d7f8b005280666c', commit_message='Upload model', commit_description='', oid='701fc782c5b9f0e23b444f2f0d7f8b005280666c', pr_url=None, pr_revision=None, pr_num=None)